---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [ ]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [1]:
import pandas as pd
import numpy as np

In [81]:
def separateStateRegionValues(x):
    global curStateName
    
    #print(x[0])
    if (x[0].endswith('[edit]')):
        curStateName = x[0].replace('[edit]','')
        return np.NAN
    else:
        regionName = x[0].split(' (')[0]
        #print({'State': curStateName, 'RegionName': regionName})
        return {'State': curStateName, 'RegionName': regionName}

def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    df = pd.read_csv('course1_downloads/university_towns.txt', sep='\n', header=None)
                                 
    
    return (df.apply(separateStateRegionValues, axis=1)
             .dropna()
             .apply(pd.Series))[['State','RegionName']]
                              

In [86]:
%%timeit -n 1 
get_list_of_university_towns()

142 ms ± 22.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
df = pd.read_excel('course1_downloads/gdplev.xls', skiprows=219, usecols='E,G', names=['Quarter','GDP'])

prevGDP = np.NAN
prevQGDPDecline = False
prevQGDPRise = True

def processrow(row):
    global prevGDP
    if (prevGDP != np.NAN):
        if row['GDP'] <= prevGDP:
            row['GDPDecline'] = True
        elif row['GDP'] > prevGDP:
            row['GDPDecline'] = False
    prevGDP = row['GDP']
    return row
df.apply(processrow,axis=1)


,Quarter,GDP,GDPDecline
0,2000q1,12359.1,True
1,2000q2,12592.5,False
2,2000q3,12607.7,False
3,2000q4,12679.3,False
4,2001q1,12643.3,True
5,2001q2,12710.3,False
6,2001q3,12670.1,True
7,2001q4,12705.3,False
8,2002q1,12822.3,False
9,2002q2,12893.0,False


In [70]:
df.shape[0]

66

In [22]:
def get_recession():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('course1_downloads/gdplev.xls', skiprows=219, usecols='E,G', names=['Quarter','GDP'])
    
    recession = False
    dfr=[]

    for index, row in df.iterrows():
        if (index == 0 or index == (df.shape[0]-1)):
            continue
        if (recession == False and(df.iloc[index-1]['GDP'] > df.iloc[index]['GDP'] > df.iloc[index+1]['GDP'])):
            recession = True
            rstart = df.iloc[index]
            #rstartIndex = index
            #print("rstart" + rstart)
        elif (recession == True and (df.iloc[index-1]['GDP'] < df.iloc[index]['GDP'] < df.iloc[index+1]['GDP'])):
            recession = False
            rend = df.iloc[index]
            #rendindex = index
            dfr.append((rstart, rend))
            #print("rend:" + rend)
    return dfr

In [23]:
get_recession()

[(Quarter     2008q3
  GDP        14891.6
  Name: 34, dtype: object, Quarter     2009q3
  GDP        14402.5
  Name: 38, dtype: object)]

In [27]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    return get_recession()[0][0].Quarter
        
get_recession_start()

'2008q3'

In [28]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    return get_recession()[0][1].Quarter

get_recession_end()

'2009q3'

In [50]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    r = get_recession()[0]
    start, end = r[0].name, r[1].name
    #print(start, end)
    
    rData = df.iloc[start:end+1]
    return rData.loc[rData['GDP'] == np.min(rData['GDP'])].iloc[0]['Quarter']

get_recession_bottom()

'2009q2'

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    return "ANSWER"

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"